In [66]:
%load_ext autoreload
%autoreload 2
import time

import osu.rulesets.beatmap as bm
import osu.rulesets.replay as rp
import numpy as np

test_name = '1hope'

test_map = f'assets/{test_name}_map.osu'
test_song = f'assets/{test_name}_song.mp3'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
# testing dt replay
real_replay = rp.load(f'assets/ringyou_mrekk_replay.osr')
real_replay_map = f'assets/ringyou_map.osu'
real_replay_song = f'assets/ringyou_song.mp3'

# testing hr replay
# real_replay = rp.load(f'assets/1hope_whitecat_replay.osr')
# real_replay_map = f'assets/1hope_map.osu'
# real_replay_song = f'assets/1hope_song.mp3'

# testing ez replay
# real_replay = rp.load(f'assets/ghosts_ezchamp_replay.osr')
# real_replay_map = f'assets/ghosts_map.osu'
# real_replay_song = f'assets/ghosts_song.mp3'

In [68]:
import importlib
import osu.preview.preview as preview

importlib.reload(preview)
# try previewing a real replay

preview.preview_replay(real_replay, real_replay_map, real_replay_song)

HAS DT
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k2 pressed
k1 pressed
k2 released
k2 pressed
k1 released
k2 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k1 pressed
k1 released
k2 pressed
k1 pressed
k2 released
k2 pressed
k1 released
k1 pressed
k2 released
k1 released
k2 p

In [50]:
# data = dataset.input_data(test_map)
# data = np.reshape(data.values, (-1, dataset.BATCH_LENGTH, len(dataset.INPUT_FEATURES)))
# data = torch.FloatTensor(data).to('cuda')

In [4]:
replay_data = np.load(f".generated/{test_name}.npy")
replay_data

array([[ 0.00259041,  0.07582808],
       [ 0.01636691,  0.15886295],
       [ 0.02021809,  0.17380756],
       ...,
       [ 0.00189539, -0.00471151],
       [ 0.00189539, -0.00471151],
       [ 0.0018954 , -0.00471151]], shape=(10240, 2), dtype=float32)

In [6]:


print(test_map)
# test_map.apply_mods(['hr', 'ht'])
preview.preview_replay_raw(replay_data, test_map, ['ez'], test_song)

assets/1hope_map.osu


In [ ]:
# alt_data = dataset.replay_to_output_data(test_map, test_replay)
# alt_data = np.concatenate(alt_data)
# 
# alt_data

In [ ]:
# actually play out the replay
import win32gui
import mouse
import time
import osu.rulesets.core as osu_core
import pyautogui

def active_win_name():
    hwnd = win32gui.GetForegroundWindow()
    return win32gui.GetWindowText(hwnd)

def osu_to_screen_pixel(x, y) -> (int, int):
    w, h = pyautogui.size()
    
    # screen/osu ratio
    # https://osu.ppy.sh/wiki/en/Client/Playfield
    osr_x = w / 640 * 0.75 # for some reason this constant just works. misleading wiki?
    osr_y = h / 480
    
    x = x * osr_x
    y = y * osr_y
    
    # osu width and height
    ow = osu_core.SCREEN_WIDTH * osr_x
    oh = osu_core.SCREEN_HEIGHT * osr_y
    
    offset_x = 0.5 * (w - ow)
    # "The playfield is slightly shifted vertically, placed 8 game pixels lower than the window's centre."
    offset_y = 0.5 * (h - oh) + (8 * osr_y)
    
    return int(x + offset_x), int(y + offset_y)

time.sleep(1.25)
win_name = active_win_name()

target_interval = 0.024  # 24ms per frame
start_time = time.perf_counter()
frame_count = 0

for x, y in replay_data:
    if win_name != active_win_name():
        break
    x = (x + 0.5) * osu_core.SCREEN_WIDTH
    y = (y + 0.5) * osu_core.SCREEN_HEIGHT
    (x, y) = osu_to_screen_pixel(x, y)
    mouse.move(x, y)
    
    frame_count += 1
    
    target_time = start_time + (frame_count * target_interval)
    current_time = time.perf_counter()
    
    # run at constant rate 
    sleep_time = target_time - current_time
    if sleep_time > 0:
        time.sleep(sleep_time)
